# BERTで文をベクトル化して類似度を求めるコード
ブログ：https://techblog.gmo-ap.jp/2022/12/21/bert_calc_sentence_similarity/

# 初期設定

## 必要なライブラリのダウンロード

In [1]:
!pip install transformers
!pip install sentence_transformers
!pip install fugashi[unidic-lite]
!pip install ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 907.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=dc9a6c63f34402e1806c5ee6587077e774b196e6980478436a93f646f6b528b0
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 10.9 

## ライブラリのimport

In [2]:
from transformers import BertJapaneseTokenizer, BertModel

from sentence_transformers import SentenceTransformer
from sentence_transformers import models

import torch
import numpy as np

## よく使う変数の定義

In [3]:
# 今回使うbertのバージョンの名前
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)

# 文のベクトル化

In [8]:
def sentence_to_vector(model, tokenizer, sentence):

  # 文を単語に区切って数字にラベル化
  tokens = tokenizer(sentence)['input_ids']

  # BERTモデルの処理のためtensor型に変換
  input = torch.tensor(tokens).reshape(1,-1)

  # BERTモデルに入力し文のベクトルを取得
  with torch.no_grad():
    outputs = model(input, output_hidden_states=True)
    last_hidden_state = outputs.last_hidden_state[0]
    averaged_hidden_state = last_hidden_state.sum(dim=0) / len(last_hidden_state)

  return averaged_hidden_state

In [11]:
sentence = "我輩は猫である。"
sentence_vector = sentence_to_vector(model, tokenizer, sentence)

# 文の類似度計算

In [12]:
def calc_similarity(sentence1, sentence2):
  print("{}\n{}".format(sentence1, sentence2))

  sentence_vector1 = sentence_to_vector(model, tokenizer, sentence1)
  sentence_vector2 = sentence_to_vector(model, tokenizer, sentence2)

  score = torch.nn.functional.cosine_similarity(sentence_vector1, sentence_vector2, dim=0).detach().numpy().copy()
  print("類似度：", score)

In [14]:
sentence1 = "吾輩は猫である"

sentence2 = "私は猫です"

calc_similarity(sentence1, sentence2)

吾輩は猫である
私は猫です
類似度： 0.85347897


In [15]:
sentence3 = "お昼ご飯にカレーが食べたい"

calc_similarity(sentence1, sentence3)

吾輩は猫である
お昼ご飯にカレーが食べたい
類似度： 0.76310676
